In [0]:
### Note that passage of data between PySpark and Pandas will be conducted through PyArrow which is already installed

In [0]:
!pip uninstall pystan

In [0]:
!pip install pystan~=2.14


     |█▉                              | 3.6 MB 2.

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 62.6 MB 91 kB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install fbprophet


     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 47 kB 8.2 MB/s 
     |████████████████████████████████| 179 kB 7.1 MB/s 
     |████████████████████████████████| 78 kB 9.0 MB/s 
     |█████████████████▊              | 3.0 MB 49.7 MB/s eta 0:00

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 5.4 MB 49.7 MB/s 
     |████████████████████████████████| 1.8 MB 33.0 MB/s 
  ERROR: Command errored out with exit status 1:
   command: /databricks/python3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-17ow6p4b/fbprophet_31992014c13d4099bf0653cc042ffb2b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-17ow6p4b/fbprophet_31992014c13d4099bf0653cc042ffb2b/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-ufp2yfs

In [0]:
!pip install altair

     |████████████████████████████████| 812 kB 5.0 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
from fbprophet import Prophet 
import pyarrow

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

import altair as alt

from pyspark.sql import SparkSession
import pyspark

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/sjtalkar/timeseries_analysis/main/spring_groundwater_levels.csv")

In [0]:
spark = (SparkSession
.builder
.master("local[*]")
.appName('Time Series application')
.getOrCreate())

sc = spark.sparkContext

In [0]:
df.head(4)

,TOWNSHIP_RANGE,YEAR,GSE_GWE
0,T01N R02E,2014,57.046154
1,T01N R02E,2015,56.027436
2,T01N R02E,2016,48.830000
3,T01N R02E,2017,48.007333


In [0]:
df['YEAR'] = pd.to_datetime(df['YEAR'], format='%Y')
df.head()

,TOWNSHIP_RANGE,YEAR,GSE_GWE
0,T01N R02E,2014-01-01,57.046154
1,T01N R02E,2015-01-01,56.027436
2,T01N R02E,2016-01-01,48.830000
3,T01N R02E,2017-01-01,48.007333
4,T01N R02E,2018-01-01,45.985000


In [0]:
print("\nMissing Values:\n",df.isnull().any())
print("\nUnique Values:\n",df.nunique())



Missing Values:
 TOWNSHIP_RANGE    False
YEAR              False
GSE_GWE           False
dtype: bool

Unique Values:
 TOWNSHIP_RANGE     478
YEAR                 8
GSE_GWE           3642
dtype: int64


In [0]:
item_df = df.copy()
item_df.head()

,TOWNSHIP_RANGE,YEAR,GSE_GWE
0,T01N R02E,2014-01-01,57.046154
1,T01N R02E,2015-01-01,56.027436
2,T01N R02E,2016-01-01,48.830000
3,T01N R02E,2017-01-01,48.007333
4,T01N R02E,2018-01-01,45.985000


In [0]:
alt.Chart(item_df[item_df['TOWNSHIP_RANGE'].isin(list(item_df.TOWNSHIP_RANGE.unique())[:60])]
).mark_line(
).encode(
    x="YEAR:T",
    y='GSE_GWE:Q',
    facet=alt.Facet('TOWNSHIP_RANGE:N', columns=6)
).properties(width=150, height=100)

In [0]:
alt.Chart(item_df[item_df['TOWNSHIP_RANGE'].isin(list(item_df.TOWNSHIP_RANGE.unique())[60:120])]
).mark_line(
).encode(
    x="YEAR:T",
    y='GSE_GWE:Q',
    facet=alt.Facet('TOWNSHIP_RANGE:N', columns=6)
).properties(width=150, height=100)

In [0]:
alt.Chart(item_df[item_df['TOWNSHIP_RANGE'].isin(list(item_df.TOWNSHIP_RANGE.unique())[120:180])]
).mark_line(
).encode(
    x="YEAR:T",
    y='GSE_GWE:Q',
    facet=alt.Facet('TOWNSHIP_RANGE:N', columns=6)
).properties(width=150, height=100)

In [0]:
alt.Chart(item_df[item_df['TOWNSHIP_RANGE'].isin(list(item_df.TOWNSHIP_RANGE.unique())[180:240])]
).mark_line(
).encode(
    x="YEAR:T",
    y='GSE_GWE:Q',
    facet=alt.Facet('TOWNSHIP_RANGE:N', columns=6)
).properties(width=150, height=100)

In [0]:
df_spark = spark.createDataFrame(df)

In [0]:
df_spark.show()

+--------------+-------------------+------------------+
|TOWNSHIP_RANGE|               YEAR|           GSE_GWE|
+--------------+-------------------+------------------+
|     T01N R02E|2014-01-01 00:00:00| 57.04615384615385|
|     T01N R02E|2015-01-01 00:00:00|  56.0274358974359|
|     T01N R02E|2016-01-01 00:00:00|48.830000000000005|
|     T01N R02E|2017-01-01 00:00:00|48.007333333333335|
|     T01N R02E|2018-01-01 00:00:00|            45.985|
|     T01N R02E|2019-01-01 00:00:00|           45.7815|
|     T01N R02E|2020-01-01 00:00:00|52.196000000000005|
|     T01N R02E|2021-01-01 00:00:00|53.193636363636365|
|     T01N R03E|2014-01-01 00:00:00| 32.91324675324675|
|     T01N R03E|2015-01-01 00:00:00| 38.45204819277109|
|     T01N R03E|2016-01-01 00:00:00|27.669066666666662|
|     T01N R03E|2017-01-01 00:00:00| 25.79804597701149|
|     T01N R03E|2018-01-01 00:00:00|24.053392857142857|
|     T01N R03E|2019-01-01 00:00:00| 24.28388888888889|
|     T01N R03E|2020-01-01 00:00:00| 24.41878787

In [0]:
df_spark.printSchema()

root
 |-- TOWNSHIP_RANGE: string (nullable = true)
 |-- YEAR: timestamp (nullable = true)
 |-- GSE_GWE: double (nullable = true)



In [0]:
df_spark.count()

Out[19]: 3824

In [0]:
df_spark.select(['TOWNSHIP_RANGE']).groupBy(['TOWNSHIP_RANGE']).agg({'TOWNSHIP_RANGE':'count'}).show()

+--------------+---------------------+
|TOWNSHIP_RANGE|count(TOWNSHIP_RANGE)|
+--------------+---------------------+
|     T02N R04E|                    8|
|     T01N R05E|                    8|
|     T03S R12E|                    8|
|     T01N R03E|                    8|
|     T03N R06E|                    8|
|     T01N R07E|                    8|
|     T01S R03E|                    8|
|     T02N R09E|                    8|
|     T02S R09E|                    8|
|     T02S R05E|                    8|
|     T02N R08E|                    8|
|     T02S R12E|                    8|
|     T01N R06E|                    8|
|     T03N R04E|                    8|
|     T01N R09E|                    8|
|     T03N R09E|                    8|
|     T02S R06E|                    8|
|     T02S R07E|                    8|
|     T02S R11E|                    8|
|     T01S R06E|                    8|
+--------------+---------------------+
only showing top 20 rows



In [0]:
df_spark.createOrReplaceTempView('groundwater_depth')

In [0]:
%sql
select * from groundwater_depth

TOWNSHIP_RANGE,YEAR,GSE_GWE
T01N R02E,2014-01-01T00:00:00.000+0000,57.04615384615385
T01N R02E,2015-01-01T00:00:00.000+0000,56.0274358974359
T01N R02E,2016-01-01T00:00:00.000+0000,48.830000000000005
T01N R02E,2017-01-01T00:00:00.000+0000,48.007333333333335
T01N R02E,2018-01-01T00:00:00.000+0000,45.985
T01N R02E,2019-01-01T00:00:00.000+0000,45.7815
T01N R02E,2020-01-01T00:00:00.000+0000,52.196000000000005
T01N R02E,2021-01-01T00:00:00.000+0000,53.193636363636365
T01N R03E,2014-01-01T00:00:00.000+0000,32.91324675324675
T01N R03E,2015-01-01T00:00:00.000+0000,38.45204819277109


In [0]:
%sql
select TOWNSHIP_RANGE, count(1) as num_entries
from groundwater_depth
group by TOWNSHIP_RANGE

TOWNSHIP_RANGE,num_entries
T02N R04E,8
T01N R05E,8
T03S R12E,8
T01N R03E,8
T03N R06E,8
T01N R07E,8
T01S R03E,8
T02N R09E,8
T02S R09E,8
T02S R05E,8


In [0]:
# Note that the column name renaming is a requirement of Facebook Prophet
sql = 'select TOWNSHIP_RANGE, YEAR as ds, GSE_GWE as y  from groundwater_depth order by TOWNSHIP_RANGE, ds;'
spark.sql(sql).show()

+--------------+-------------------+------------------+
|TOWNSHIP_RANGE|                 ds|                 y|
+--------------+-------------------+------------------+
|     T01N R02E|2014-01-01 00:00:00| 57.04615384615385|
|     T01N R02E|2015-01-01 00:00:00|  56.0274358974359|
|     T01N R02E|2016-01-01 00:00:00|48.830000000000005|
|     T01N R02E|2017-01-01 00:00:00|48.007333333333335|
|     T01N R02E|2018-01-01 00:00:00|            45.985|
|     T01N R02E|2019-01-01 00:00:00|           45.7815|
|     T01N R02E|2020-01-01 00:00:00|52.196000000000005|
|     T01N R02E|2021-01-01 00:00:00|53.193636363636365|
|     T01N R03E|2014-01-01 00:00:00| 32.91324675324675|
|     T01N R03E|2015-01-01 00:00:00| 38.45204819277109|
|     T01N R03E|2016-01-01 00:00:00|27.669066666666662|
|     T01N R03E|2017-01-01 00:00:00| 25.79804597701149|
|     T01N R03E|2018-01-01 00:00:00|24.053392857142857|
|     T01N R03E|2019-01-01 00:00:00| 24.28388888888889|
|     T01N R03E|2020-01-01 00:00:00| 24.41878787

In [0]:
df_spark.explain()

== Physical Plan ==
*(1) Scan ExistingRDD arrow[TOWNSHIP_RANGE#28,YEAR#29,GSE_GWE#30]




In [0]:
df_spark.rdd.getNumPartitions()

Out[24]: 8

In [0]:
tr_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism,['TOWNSHIP_RANGE'])).cache()

In [0]:
tr_part.explain()

== Physical Plan ==
InMemoryTableScan [TOWNSHIP_RANGE#28, ds#128, y#129]
   +- InMemoryRelation [TOWNSHIP_RANGE#28, ds#128, y#129], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange hashpartitioning(TOWNSHIP_RANGE#28, 8), REPARTITION_BY_NUM, [id=#326]
            +- *(1) Project [TOWNSHIP_RANGE#28, YEAR#29 AS ds#128, GSE_GWE#30 AS y#129]
               +- *(1) Scan ExistingRDD arrow[TOWNSHIP_RANGE#28,YEAR#29,GSE_GWE#30]




In [0]:
### Create a UDF to use FB Prophet
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType

result_schema = StructType([
    StructField('ds', TimestampType()),
    StructField('TOWNSHIP_RANGE', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat', DoubleType()),
    StructField('yhat_upper', DoubleType()),
    StructField('yhat_lower', DoubleType()),

])

In [0]:
@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def forecast_depth(tr_pd):
    model = (
            Prophet(
            interval_width=0.95, #Confidence interval
            seasonality_mode='multiplicative',
            #weekly_seasonality=True,
            yearly_seasonality=True
            )
    )
    
    model.fit(tr_pd)
    
    future_pd = model.make_future_dataframe(
            periods=2,
            freq='y' 
    )    
    
    forecast_pd = model.predict(future_pd)
    f_pd = forecast_pd [['ds', 'yhat', 'yhat_upper', 'yhat_lower']].set_index('ds')
    st_pd = tr_pd[['ds', 'TOWNSHIP_RANGE', 'y']].set_index('ds')
    
    results_pd = f_pd.join (st_pd, how='left')
    results_pd.reset_index(level=0, inplace=True)
        
    results_pd['TOWNSHIP_RANGE'] = tr_pd['TOWNSHIP_RANGE'].iloc[0]    
        
    return results_pd  [['ds', 'TOWNSHIP_RANGE', 'y', 'yhat', 'yhat_upper', 'yhat_lower']] 
          

In [0]:
from pyspark.sql.functions import current_date
results = (
        tr_part
        .groupBy (['TOWNSHIP_RANGE'])
        .apply(forecast_depth)
        .withColumn('training_date', current_date())
)

/databricks/spark/python/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [0]:
results.cache()

Out[52]: DataFrame[ds: timestamp, TOWNSHIP_RANGE: string, y: double, yhat: double, yhat_upper: double, yhat_lower: double, training_date: date]

In [0]:
results.show()

+-------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|                 ds|TOWNSHIP_RANGE|                 y|              yhat|        yhat_upper|        yhat_lower|training_date|
+-------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|2014-01-01 00:00:00|     T01N R10E|109.33333333333331|103.92359979573544| 126.3272137681425| 80.65390772412988|   2022-08-09|
|2015-01-01 00:00:00|     T01N R10E|             123.6| 119.4778833074325|141.97878493851513| 97.24197895465568|   2022-08-09|
|2016-01-01 00:00:00|     T01N R10E|126.15916666666668|134.39513639205708|156.67536728581956|112.49703180649914|   2022-08-09|
|2017-01-01 00:00:00|     T01N R10E|130.64142857142858|133.16055466276887|154.24945074934237|112.08617969294039|   2022-08-09|
|2018-01-01 00:00:00|     T01N R10E|           134.215| 151.0185084371384|171.94734481297073|129.01358110246306

In [0]:
final_df = results.toPandas()

In [0]:
final_df = final_df.set_index('ds')

In [0]:
final_df.query('TOWNSHIP_RANGE=29')[['y', 'yhat']].plot()